## AI HUB 경상도 사투리 음성 데이터 전처리


AI HUB 경상도 사투리 데이터는 음성 원천 데이터인 .wav 형식 파일과 원천 데이터의 대본인 텍스트, 해당 텍스트에 대한 음성파일에서의 화자의 발화 시간 등의 음성 파일에 대한 레이블링 정보가 담긴 JSON 형식 파일로 구성되어 있다.

연구 초반엔 1)의 방식을 통해 전처리해 학습을 진행하였으나 한 화자가 길게 발화하는 대화의 경우, 한 음성 파일 당 길이가 너무 길어져 최종적으로는 2)의 전처리 방식을 사용하여 데이터 전처리 후 학습을 진행하였다.

[JSON 파일 형식 예시]
```python
    "utterance": [
        {
            "id": "DKSR20000061.1.1.1",
            "form": "(따른)/(다른) 나라에서 먹었던 음식 중에서",
            "standard_form": "다른 나라에서 먹었던 음식 중에서",
            "dialect_form": "따른 나라에서 먹었던 음식 중에서",
            "speaker_id": "1",
            "start": 0.01,
            "end": 5.55,
            "note": "",
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/data/New_Sample/')

In [ ]:
import numpy as np
import librosa
import soundfile as sf

In [ ]:
!pip install pydub

#### 1) 화자가 변경될 때를 기준으로 음성 파일 자르는 방식

In [ ]:
import json

processed_filename = []

for f in os.listdir('label/'):
  file_name = f.split('.')[0]
  processed_filename.append(file_name)
  json_file = 'label/'+ file_name +'.json'
  with open(json_file, 'r', encoding='UTF-8') as f:
    json_data = json.load(f)

  speaker_id = 1
  speaker1_sex = json_data['speaker'][0]['sex']
  speaker2_sex = json_data['speaker'][1]['sex']
  audio_start = 0.01
  audio_end = 0.01
  audio_file = 'wavs/'+ file_name +'.wav'
  save_path = 'processed_data/'

  for i in range(0,len(json_data['utterance'])):
    if speaker_id == json_data['utterance'][i]['speaker_id']:
      continue
    else:
      speaker_id = json_data['utterance'][i]['speaker_id']
      audio_end = json_data['utterance'][i]['start']
      if speaker_id == 1:
        if speaker1_sex == "여성":
          save_file = save_path + '/speaker_' + 'woman' +'/'+file_name +'_'+str(i)+'.wav'
        else:
          save_file = save_path + '/speaker_' + 'man' +'/'+file_name +'_'+str(i)+'.wav'
      else:
        if speaker2_sex == "여성":
          save_file = save_path + '/speaker_' + 'woman' +'/'+file_name +'_'+str(i)+'.wav'
        else:
          save_file = save_path + '/speaker_' + 'man' +'/'+file_name +'_'+str(i)+'.wav'
      y, sr = librosa.load(audio_file, sr=22050)
      edited_data = split_wav(y, sr, audio_start, audio_end)
      sf.write(save_file, edited_data, sr, subtype='PCM_16')
      audio_start = audio_end
  os.remove(audio_file)
  os.remove(json_file)

#### 2) 레이블링 되어있는 문장 단위로 음성 파일 자르는 방식

In [ ]:
import json

wavs_dir = "filelists/wavs/"
processed_filename = []

for f in os.listdir('/content/drive/MyDrive/Colab Notebooks/data/New_Sample/label'):
  file_name = f.split('.')[0]
  processed_filename.append(file_name)
  json_file = 'label/'+ file_name +'.json'
  with open(json_file, 'r', encoding='UTF-8') as f:
    json_data = json.load(f)


  speaker1_sex = json_data['speaker'][0]['sex']
  speaker2_sex = json_data['speaker'][1]['sex']
  audio_file = 'wavs/'+ file_name +'.wav'
  save_path = 'processed_data/'


  y, sr = librosa.load(audio_file, sr=22050)

  for i in range(0,len(json_data['utterance'])):
    speaker_id = json_data['utterance'][i]['speaker_id']
    sentence = json_data['utterance'][i]['standard_form']
    audio_start = json_data['utterance'][i]['start']
    audio_end = json_data['utterance'][i]['end']

    if speaker_id == "1":
      if speaker1_sex == "여성":
        save_file = save_path + '/speaker_woman/wavs/'+file_name +'_'+str(i)+'.wav'
        sex = 2
      else:
        save_file = save_path + '/speaker_man/wavs/'+file_name +'_'+str(i)+'.wav'
        sex = 1
    else:
      if speaker2_sex == "여성":
        save_file = save_path + '/speaker_woman/wavs/'+file_name +'_'+str(i)+'.wav'
        sex = 2
      else:
        save_file = save_path + '/speaker_man/wavs/'+file_name +'_'+str(i)+'.wav'
        sex = 1
    edited_data = split_wav(y, sr, audio_start, audio_end)
    sf.write(save_file, edited_data, sr, subtype='PCM_16')

    if sex == 1:
        with open(save_path+"speaker_man/metadata.txt", 'a', encoding='utf8') as f1:
            f1.write(f'{wavs_dir}{save_file.split("/")[-1]}|{sentence}\n') # filelists/wavs/filename.wav|sentence
    else:
        with open(save_path+"speaker_woman/metadata.txt", 'a', encoding='utf8') as f1:
            f1.write(f'{wavs_dir}{save_file.split("/")[-1]}|{sentence}\n')